In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from dmba import classificationSummary

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pylab as plt
import numpy as np 
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from sklearn.ensemble  import VotingClassifier
from sklearn.metrics import accuracy_score

In [44]:
test_merge_data = pd.read_csv('test_merge_data.csv')
pd.set_option('display.max_columns',None)
test_merge_data

,Unnamed: 0,id,date,store_nbr,family,onpromotion,dcoilwtico,type_x,locale,locale_name,description,transferred,transactions,city,state,type_y,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay
0,0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,Empty,Empty,Empty,Empty,Empty,1448.259579,Quito,Pichincha,D,13,2017,8,0,2,1688,False
1,1,3000889,2017-08-16,1,BABY CARE,0,46.80,Empty,Empty,Empty,Empty,Empty,1448.259579,Quito,Pichincha,D,13,2017,8,0,2,1688,False
2,2,3000890,2017-08-16,1,BEAUTY,2,46.80,Empty,Empty,Empty,Empty,Empty,1448.259579,Quito,Pichincha,D,13,2017,8,0,2,1688,False
3,3,3000891,2017-08-16,1,BEVERAGES,20,46.80,Empty,Empty,Empty,Empty,Empty,1448.259579,Quito,Pichincha,D,13,2017,8,0,2,1688,False
4,4,3000892,2017-08-16,1,BOOKS,0,46.80,Empty,Empty,Empty,Empty,Empty,1448.259579,Quito,Pichincha,D,13,2017,8,0,2,1688,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,28507,3029395,2017-08-31,9,POULTRY,1,47.26,Empty,Empty,Empty,Empty,Empty,1986.976875,Quito,Pichincha,B,6,2017,8,0,3,1703,False
28508,28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,Empty,Empty,Empty,Empty,Empty,1986.976875,Quito,Pichincha,B,6,2017,8,0,3,1703,False
28509,28509,3029397,2017-08-31,9,PRODUCE,1,47.26,Empty,Empty,Empty,Empty,Empty,1986.976875,Quito,Pichincha,B,6,2017,8,0,3,1703,False
28510,28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,Empty,Empty,Empty,Empty,Empty,1986.976875,Quito,Pichincha,B,6,2017,8,0,3,1703,False


In [2]:
train_merge_data = pd.read_csv('train_merge_data.csv')
pd.set_option('display.max_columns',None)

C:\Users\rhskr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [46]:
train_merge_data['workday'] = 1 #true: 1 ,false: 0 

train_merge_data.loc[train_merge_data.Weekday > 4, 'workday'] = 0
train_merge_data.loc[train_merge_data.type_x == 'Bridge'  , 'workday'] = 0
train_merge_data.loc[train_merge_data.type_x == 'Work Day', 'workday'] = 1
train_merge_data.loc[train_merge_data.type_x == 'Transfer', 'workday'] = 0
train_merge_data.loc[(train_merge_data.type_x == 'Holiday') & (train_merge_data.transferred == False), 'workday'] = 0
train_merge_data.loc[(train_merge_data.type_x == 'Holiday') & (train_merge_data.transferred == True ), 'workday'] = 1

#family 변수 묶어주기

train_merge_data['family'].replace(['BREAD/BAKERY','DAIRY'], 'num1', inplace=True)
train_merge_data['family'].replace(['AUTOMOTIVE','BABY CARE','BEAUTY','BOOKS','CELEBRATION','FROZEN FOODS','GROCERY II','HARDWARE','HOME AND KITCHEN I', 'HOME AND KITCHEN II',
                                   'HOME APPLIANCES','HOME CARE','LADIESWEAR','LAWN AND GARDEN','LINGERIE','LIQUOR,WINE,BEER','MAGAZINES',
                                   'PET SUPPLIES','PLAYERS AND ELECTRONICS','PREPARED FOODS','PRODUCE','SCHOOL AND OFFICE SUPPLIES','SEAFOOD'], 'num2', inplace=True)
train_merge_data['family'].replace(['Tools','PERSONAL CARE','MEATS','POULTRY','DELI','WINE,BEER','LIQUOR', 'EGGS'], 'num3', inplace=True)
train_merge_data['family'].replace(['BEVERAGES'], 'num4', inplace=True)
train_merge_data['family'].replace(['CLEANING'], 'num5', inplace=True)
train_merge_data['family'].replace(['GROCERY I'], 'num6', inplace=True)

In [47]:
#get dummies
train_merge_data.loc[train_merge_data['description'].str.contains('Terremoto', na=False),
                 'description'] = 'Earthquake'
train_merge_data.loc[train_merge_data['description'].str.contains('futbol', na=False), 
                 'description'] = 'Football'

train_merge_data = pd.get_dummies(train_merge_data,columns=['family','type_x','locale','type_y']) 

descriptions = pd.get_dummies(train_merge_data['description'])[['Earthquake', 'Cyber Monday', 'Black Friday']]
train_merge_data = train_merge_data.merge(descriptions, how='left', left_index=True, right_index=True)

#이진형
train_merge_data['NewYearsDay'] = np.where(train_merge_data['NewYearsDay'] == 'True',1,0)

In [48]:
#버릴 변수
train_merge_data.drop(columns=['Unnamed: 0','id','description','locale_name','transferred','city','state'],inplace=True)

In [49]:
pd.set_option('display.max_columns',None)
train_merge_data

,date,store_nbr,sales,onpromotion,dcoilwtico,transactions,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,workday,family_num1,family_num2,family_num3,family_num4,family_num5,family_num6,type_x_Additional,type_x_Bridge,type_x_Empty,type_x_Event,type_x_Holiday,type_x_Transfer,type_x_Work Day,locale_Empty,locale_Local,locale_National,locale_Regional,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,Earthquake,Cyber Monday,Black Friday
0,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,438.133,0,47.57,2155.000000,6,2017,8,0,1,1687,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3054344,2017-08-15,9,154.553,1,47.57,2155.000000,6,2017,8,0,1,1687,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3054345,2017-08-15,9,2419.729,148,47.57,2155.000000,6,2017,8,0,1,1687,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3054346,2017-08-15,9,121.000,8,47.57,2155.000000,6,2017,8,0,1,1687,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [50]:
train_merge_data

,date,store_nbr,sales,onpromotion,dcoilwtico,transactions,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,workday,family_num1,family_num2,family_num3,family_num4,family_num5,family_num6,type_x_Additional,type_x_Bridge,type_x_Empty,type_x_Event,type_x_Holiday,type_x_Transfer,type_x_Work Day,locale_Empty,locale_Local,locale_National,locale_Regional,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,Earthquake,Cyber Monday,Black Friday
0,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,2013-01-01,1,0.000,0,93.14,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,438.133,0,47.57,2155.000000,6,2017,8,0,1,1687,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3054344,2017-08-15,9,154.553,1,47.57,2155.000000,6,2017,8,0,1,1687,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3054345,2017-08-15,9,2419.729,148,47.57,2155.000000,6,2017,8,0,1,1687,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3054346,2017-08-15,9,121.000,8,47.57,2155.000000,6,2017,8,0,1,1687,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [51]:
#train,valid나누기
sub_train_data = train_merge_data[train_merge_data.date < '2016-08-01']
valid_data = train_merge_data[train_merge_data.date >= '2016-08-01']

In [52]:
# 검증 과정에서 사용할 데이터
sub_train_data.drop(columns = ['date'], inplace = True)
valid_data.drop(columns = ['date'], inplace = True)

# 최종 모델 훈련에서 사용할 데이터
train_merge_data.drop(columns = ['date'], inplace = True)
test_merge_data.drop(columns = ['date'], inplace = True)

X_sub_train_data = sub_train_data.drop(columns = ['sales'])
y_sub_train_data = sub_train_data['sales']

X_valid_data = valid_data.drop(columns = ['sales'])
y_valid_data = valid_data['sales']

C:\Users\rhskr\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [53]:
X_sub_train_data = pd.get_dummies(X_sub_train_data,columns=['store_nbr']) 
X_valid_data = pd.get_dummies(X_valid_data,columns=['store_nbr'])
X_valid_data

,onpromotion,dcoilwtico,transactions,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,workday,family_num1,family_num2,family_num3,family_num4,family_num5,family_num6,type_x_Additional,type_x_Bridge,type_x_Empty,type_x_Event,type_x_Holiday,type_x_Transfer,type_x_Work Day,locale_Empty,locale_Local,locale_National,locale_Regional,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,Earthquake,Cyber Monday,Black Friday,store_nbr_1,store_nbr_2,store_nbr_3,store_nbr_4,store_nbr_5,store_nbr_6,store_nbr_7,store_nbr_8,store_nbr_9,store_nbr_10,store_nbr_11,store_nbr_12,store_nbr_13,store_nbr_14,store_nbr_15,store_nbr_16,store_nbr_17,store_nbr_18,store_nbr_19,store_nbr_20,store_nbr_21,store_nbr_22,store_nbr_23,store_nbr_24,store_nbr_25,store_nbr_26,store_nbr_27,store_nbr_28,store_nbr_29,store_nbr_30,store_nbr_31,store_nbr_32,store_nbr_33,store_nbr_34,store_nbr_35,store_nbr_36,store_nbr_37,store_nbr_38,store_nbr_39,store_nbr_40,store_nbr_41,store_nbr_42,store_nbr_43,store_nbr_44,store_nbr_45,store_nbr_46,store_nbr_47,store_nbr_48,store_nbr_49,store_nbr_50,store_nbr_51,store_nbr_52,store_nbr_53,store_nbr_54
2368278,0,40.05,1750.0,13,2016,8,0,0,1308,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368279,0,40.05,1750.0,13,2016,8,0,0,1308,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368280,0,40.05,1750.0,13,2016,8,0,0,1308,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368281,10,40.05,1750.0,13,2016,8,0,0,1308,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368282,0,40.05,1750.0,13,2016,8,0,0,1308,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,0,47.57,2155.0,6,2017,8,0,1,1687,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3054344,1,47.57,2155.0,6,2017,8,0,1,1687,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3054345,148,47.57,2155.0,6,2017,8,0,1,1687,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3054346,8,47.57,2155.0,6,2017,8,0,1,1687,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
X_sub_train_data

,onpromotion,dcoilwtico,transactions,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,workday,family_num1,family_num2,family_num3,family_num4,family_num5,family_num6,type_x_Additional,type_x_Bridge,type_x_Empty,type_x_Event,type_x_Holiday,type_x_Transfer,type_x_Work Day,locale_Empty,locale_Local,locale_National,locale_Regional,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,Earthquake,Cyber Monday,Black Friday,store_nbr_1,store_nbr_2,store_nbr_3,store_nbr_4,store_nbr_5,store_nbr_6,store_nbr_7,store_nbr_8,store_nbr_9,store_nbr_10,store_nbr_11,store_nbr_12,store_nbr_13,store_nbr_14,store_nbr_15,store_nbr_16,store_nbr_17,store_nbr_18,store_nbr_19,store_nbr_20,store_nbr_21,store_nbr_22,store_nbr_23,store_nbr_24,store_nbr_25,store_nbr_26,store_nbr_27,store_nbr_28,store_nbr_29,store_nbr_30,store_nbr_31,store_nbr_32,store_nbr_33,store_nbr_34,store_nbr_35,store_nbr_36,store_nbr_37,store_nbr_38,store_nbr_39,store_nbr_40,store_nbr_41,store_nbr_42,store_nbr_43,store_nbr_44,store_nbr_45,store_nbr_46,store_nbr_47,store_nbr_48,store_nbr_49,store_nbr_50,store_nbr_51,store_nbr_52,store_nbr_53,store_nbr_54
0,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368273,1,40.546667,2115.000000,6,2016,7,1,6,1307,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368274,1,40.546667,2115.000000,6,2016,7,1,6,1307,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368275,3,40.546667,2115.000000,6,2016,7,1,6,1307,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368276,0,40.546667,2115.000000,6,2016,7,1,6,1307,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
X_sub = X_sub_train_data[['onpromotion', 'dcoilwtico', 'transactions']]
X_valid = X_valid_data[['onpromotion', 'dcoilwtico', 'transactions']]

scaler = StandardScaler()
scaler.fit(X_sub)
label = ['onpromotionn','dcoilwticoo','transactionss']
X_sub_trans = pd.DataFrame(scaler.transform(X_sub),columns=label)
X_valid_trans = pd.DataFrame(scaler.transform(X_valid),columns=label)

X_sub_train_data_trans = pd.concat([X_sub_trans,X_sub_train_data],axis=1)
X_valid_data_trans = pd.concat([X_valid_trans,X_valid_data],axis=1)


In [63]:
X_sub_train_data_trans

,onpromotionn,dcoilwticoo,transactionss,onpromotion,dcoilwtico,transactions,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,workday,family_num1,family_num2,family_num3,family_num4,family_num5,family_num6,type_x_Additional,type_x_Bridge,type_x_Empty,type_x_Event,type_x_Holiday,type_x_Transfer,type_x_Work Day,locale_Empty,locale_Local,locale_National,locale_Regional,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,Earthquake,Cyber Monday,Black Friday,store_nbr_1,store_nbr_2,store_nbr_3,store_nbr_4,store_nbr_5,store_nbr_6,store_nbr_7,store_nbr_8,store_nbr_9,store_nbr_10,store_nbr_11,store_nbr_12,store_nbr_13,store_nbr_14,store_nbr_15,store_nbr_16,store_nbr_17,store_nbr_18,store_nbr_19,store_nbr_20,store_nbr_21,store_nbr_22,store_nbr_23,store_nbr_24,store_nbr_25,store_nbr_26,store_nbr_27,store_nbr_28,store_nbr_29,store_nbr_30,store_nbr_31,store_nbr_32,store_nbr_33,store_nbr_34,store_nbr_35,store_nbr_36,store_nbr_37,store_nbr_38,store_nbr_39,store_nbr_40,store_nbr_41,store_nbr_42,store_nbr_43,store_nbr_44,store_nbr_45,store_nbr_46,store_nbr_47,store_nbr_48,store_nbr_49,store_nbr_50,store_nbr_51,store_nbr_52,store_nbr_53,store_nbr_54
0,-0.147314,0.741502,-0.269291,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-0.147314,0.741502,-0.269291,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.147314,0.741502,-0.269291,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-0.147314,0.741502,-0.269291,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-0.147314,0.741502,-0.269291,0,93.140000,1420.876311,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368273,-0.040527,-1.233201,0.468437,1,40.546667,2115.000000,6,2016,7,1,6,1307,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368274,-0.040527,-1.233201,0.468437,1,40.546667,2115.000000,6,2016,7,1,6,1307,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368275,0.173047,-1.233201,0.468437,3,40.546667,2115.000000,6,2016,7,1,6,1307,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368276,-0.147314,-1.233201,0.468437,0,40.546667,2115.000000,6,2016,7,1,6,1307,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
X_sub_train_data_trans.drop(columns = ['onpromotion','dcoilwtico','transactions'],inplace=True)

In [65]:
X_sub_train_data_trans

,onpromotionn,dcoilwticoo,transactionss,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,workday,family_num1,family_num2,family_num3,family_num4,family_num5,family_num6,type_x_Additional,type_x_Bridge,type_x_Empty,type_x_Event,type_x_Holiday,type_x_Transfer,type_x_Work Day,locale_Empty,locale_Local,locale_National,locale_Regional,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,Earthquake,Cyber Monday,Black Friday,store_nbr_1,store_nbr_2,store_nbr_3,store_nbr_4,store_nbr_5,store_nbr_6,store_nbr_7,store_nbr_8,store_nbr_9,store_nbr_10,store_nbr_11,store_nbr_12,store_nbr_13,store_nbr_14,store_nbr_15,store_nbr_16,store_nbr_17,store_nbr_18,store_nbr_19,store_nbr_20,store_nbr_21,store_nbr_22,store_nbr_23,store_nbr_24,store_nbr_25,store_nbr_26,store_nbr_27,store_nbr_28,store_nbr_29,store_nbr_30,store_nbr_31,store_nbr_32,store_nbr_33,store_nbr_34,store_nbr_35,store_nbr_36,store_nbr_37,store_nbr_38,store_nbr_39,store_nbr_40,store_nbr_41,store_nbr_42,store_nbr_43,store_nbr_44,store_nbr_45,store_nbr_46,store_nbr_47,store_nbr_48,store_nbr_49,store_nbr_50,store_nbr_51,store_nbr_52,store_nbr_53,store_nbr_54
0,-0.147314,0.741502,-0.269291,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-0.147314,0.741502,-0.269291,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.147314,0.741502,-0.269291,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-0.147314,0.741502,-0.269291,13,2013,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-0.147314,0.741502,-0.269291,13,2013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368273,-0.040527,-1.233201,0.468437,6,2016,7,1,6,1307,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368274,-0.040527,-1.233201,0.468437,6,2016,7,1,6,1307,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368275,0.173047,-1.233201,0.468437,6,2016,7,1,6,1307,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2368276,-0.147314,-1.233201,0.468437,6,2016,7,1,6,1307,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# LinearRegression

In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

model = LinearRegression()
model.fit(X_sub_train_data_trans, y_sub_train_data)
train_pred = model.predict(X_sub_train_data_trans)
train_pred[train_pred < 0] = 0

print(mean_squared_log_error(y_sub_train_data, train_pred))

6.732955742871608


# random forest 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfmodel = RandomForestRegressor(min_samples_split = 10)
rfmodel.fit(X_sub_train_data_trans, y_sub_train_data)
train_pred = rfmodel.predict(X_sub_train_data_trans)
train_pred[train_pred < 0] = 0

print(mean_squared_log_error(y_sub_train_data, train_pred))

# test

In [4]:
X_train_data = train_merge_data.drop(columns = ['sales'])
y_train_data = train_merge_data['sales']
test_data = pd.read_csv('test_merge_data.csv')

In [5]:
X_train_data['workday'] = 1 #true: 1 ,false: 0 

X_train_data.loc[X_train_data.Weekday > 4, 'workday'] = 0
X_train_data.loc[X_train_data.type_x == 'Bridge'  , 'workday'] = 0
X_train_data.loc[X_train_data.type_x == 'Work Day', 'workday'] = 1
X_train_data.loc[X_train_data.type_x == 'Transfer', 'workday'] = 0
X_train_data.loc[(X_train_data.type_x == 'Holiday') & (train_merge_data.transferred == False), 'workday'] = 0
X_train_data.loc[(X_train_data.type_x == 'Holiday') & (train_merge_data.transferred == True ), 'workday'] = 1

X_train_data['family'].replace(['BREAD/BAKERY','DAIRY'], 'num1', inplace=True)
X_train_data['family'].replace(['AUTOMOTIVE','BABY CARE','BEAUTY','BOOKS','CELEBRATION','FROZEN FOODS','GROCERY II','HARDWARE','HOME AND KITCHEN I', 'HOME AND KITCHEN II',
                                   'HOME APPLIANCES','HOME CARE','LADIESWEAR','LAWN AND GARDEN','LINGERIE','LIQUOR,WINE,BEER','MAGAZINES',
                                   'PET SUPPLIES','PLAYERS AND ELECTRONICS','PREPARED FOODS','PRODUCE','SCHOOL AND OFFICE SUPPLIES','SEAFOOD'], 'num2', inplace=True)
X_train_data['family'].replace(['Tools','PERSONAL CARE','MEATS','POULTRY','DELI','WINE,BEER','LIQUOR', 'EGGS'], 'num3', inplace=True)
X_train_data['family'].replace(['BEVERAGES'], 'num4', inplace=True)
X_train_data['family'].replace(['CLEANING'], 'num5', inplace=True)
X_train_data['family'].replace(['GROCERY I'], 'num6', inplace=True)




test_data['family'].replace(['BREAD/BAKERY','DAIRY'], 'num1', inplace=True)
test_data['family'].replace(['AUTOMOTIVE','BABY CARE','BEAUTY','BOOKS','CELEBRATION','FROZEN FOODS','GROCERY II','HARDWARE','HOME AND KITCHEN I', 'HOME AND KITCHEN II',
                                   'HOME APPLIANCES','HOME CARE','LADIESWEAR','LAWN AND GARDEN','LINGERIE','LIQUOR,WINE,BEER','MAGAZINES',
                                   'PET SUPPLIES','PLAYERS AND ELECTRONICS','PREPARED FOODS','PRODUCE','SCHOOL AND OFFICE SUPPLIES','SEAFOOD'], 'num2', inplace=True)
test_data['family'].replace(['Tools','PERSONAL CARE','MEATS','POULTRY','DELI','WINE,BEER','LIQUOR', 'EGGS'], 'num3', inplace=True)
test_data['family'].replace(['BEVERAGES'], 'num4', inplace=True)
test_data['family'].replace(['CLEANING'], 'num5', inplace=True)
test_data['family'].replace(['GROCERY I'], 'num6', inplace=True)



In [20]:
X_train = X_train_data[['onpromotion', 'dcoilwtico', 'transactions']]
X_valid = test_data[['onpromotion','dcoilwtico','transactions']]

scaler = StandardScaler()
scaler.fit(X_train)
label = ['onpromotionn','dcoilwticoo','transactionss']
X_train_trans = pd.DataFrame(scaler.transform(X_train),columns=label)
X_valid_trans = pd.DataFrame(scaler.transform(X_valid),columns=label)

X_train_data_trans = pd.concat([X_train_trans,X_train_data],axis=1)
X_valid_data_trans = pd.concat([X_valid_trans,test_data],axis=1)

X_train_data_trans.drop(columns = ['onpromotion','dcoilwtico','transactions'],inplace=True)
X_valid_data_trans.drop(columns = ['onpromotion','dcoilwtico','transactions'],inplace=True)

In [23]:
X_train_data_trans.drop(columns=['Unnamed: 0','id','description','locale_name','transferred','city','state'],inplace=True)
X_valid_data_trans.drop(columns=['Unnamed: 0','id','description','locale_name','transferred','city','state'],inplace=True)

In [32]:
X_valid_data_trans['NewYearsDay'].value_counts()

False    28512
Name: NewYearsDay, dtype: int64

In [34]:
#get dummies
X_train_data_trans = pd.get_dummies(X_train_data_trans,columns=['family','type_x','locale','type_y']) 
X_valid_data_trans = pd.get_dummies(X_valid_data_trans,columns=['family','type_x','locale','type_y']) 

In [37]:
#이진형
X_train_data_trans['NewYearsDay'] = np.where(X_train_data_trans['NewYearsDay'] == 'True',1,0)
X_valid_data_trans['NewYearsDay'] = np.where(X_valid_data_trans['NewYearsDay'] == 'True',1,0)

In [41]:
X_train_data_trans.drop(columns = ['date','description'],inplace=True)

In [43]:
X_valid_data_trans = pd.get_dummies(X_valid_data_trans,columns=['family','type_x','locale','type_y']) 

KeyError: "None of [Index(['family', 'type_x', 'locale', 'type_y'], dtype='object')] are in the [columns]"

In [44]:
X_valid_data_trans.drop(columns=['date'],inplace=True)

In [52]:
x_train_data_trans = X_train_data_trans.drop(columns=['type_x_Additional','type_x_Bridge','type_x_Event','type_x_Holiday','type_x_Transfer',
                                                     'locale_National','locale_Regional','workday'])

In [46]:
X_valid_data_trans

,onpromotionn,dcoilwticoo,transactionss,store_nbr,cluster,Year,Month,Weekend,Weekday,trend,NewYearsDay,family_num1,family_num2,family_num3,family_num4,family_num5,family_num6,type_x_Empty,type_x_Holiday,locale_Empty,locale_Local,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E
0,-0.213586,-0.818712,-0.238448,1,13,2017,8,0,2,1688,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
1,-0.213586,-0.818712,-0.238448,1,13,2017,8,0,2,1688,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
2,-0.050386,-0.818712,-0.238448,1,13,2017,8,0,2,1688,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
3,1.418410,-0.818712,-0.238448,1,13,2017,8,0,2,1688,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0
4,-0.213586,-0.818712,-0.238448,1,13,2017,8,0,2,1688,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,-0.131986,-0.800799,0.331280,9,6,2017,8,0,3,1703,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0
28508,-0.213586,-0.800799,0.331280,9,6,2017,8,0,3,1703,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0
28509,-0.131986,-0.800799,0.331280,9,6,2017,8,0,3,1703,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0
28510,0.520812,-0.800799,0.331280,9,6,2017,8,0,3,1703,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0


In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(min_samples_split = 10)
rfmodel.fit(x_train_data_trans[-50000:], y_train_data[-50000:])

RandomForestRegressor(min_samples_split=10)

In [54]:
test_pred = rfmodel.predict(X_valid_data_trans)
test_pred[test_pred < 0] = 0

In [55]:
submission = pd.read_csv('sample_submission.csv')
submission['sales'] = test_pred

In [56]:
submission

,id,sales
0,3000888,14.371656
1,3000889,14.371656
2,3000890,31.723189
3,3000891,2337.168070
4,3000892,14.371656
...,...,...
28507,3029395,247.960242
28508,3029396,13.100353
28509,3029397,55.699273
28510,3029398,416.277328


In [57]:
submission.to_csv('submission.csv', index = False)